# 📊 Análise de Séries Temporais com RNN

Este notebook demonstra como usar Redes Neurais Recorrentes (RNN) para previsão de séries temporais, especificamente para dados de ações.

## 🎯 Objetivos
- Carregar e analisar dados de séries temporais
- Preparar dados para treinamento de modelos RNN
- Treinar modelos LSTM e GRU
- Avaliar performance dos modelos
- Fazer previsões futuras

## 📚 Bibliotecas Utilizadas
- **PyTorch**: Para implementação das redes neurais
- **yfinance**: Para download de dados de ações
- **pandas & numpy**: Para manipulação de dados
- **matplotlib & seaborn**: Para visualizações
- **scikit-learn**: Para métricas de avaliação

In [ ]:
# Importar bibliotecas necessárias
import sys
import os

# Adicionar o diretório src ao path
current_dir = os.path.dirname(os.path.abspath('.'))
src_dir = os.path.join(current_dir, 'src')
sys.path.append(src_dir)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuração de estilo
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Importar módulos locais
from data_loader import TimeSeriesDataLoader
from model import create_model
from trainer import TimeSeriesTrainer
from utils import (
    plot_training_history, 
    plot_predictions, 
    plot_interactive_predictions,
    plot_future_predictions,
    create_performance_report,
    plot_model_comparison,
    analyze_residuals
)

print("✅ Bibliotecas importadas com sucesso!")

## 1. 📊 Carregamento e Análise de Dados

Primeiro, vamos carregar dados de uma ação e fazer uma análise exploratória.

In [ ]:
# Configurações iniciais
symbol = 'AAPL'  # Símbolo da ação
period = '2y'    # Período de dados

print(f"🚀 Carregando dados para {symbol}...")

# Inicializar carregador
loader = TimeSeriesDataLoader()

# Tentar baixar dados reais, se falhar usar dados sintéticos
try:
    data = loader.download_stock_data(symbol, period)
    print(f"✅ Dados reais baixados com sucesso!")
except Exception as e:
    print(f"⚠️ Erro ao baixar dados reais: {e}")
    print("🔄 Usando dados sintéticos para demonstração...")
    data = loader.generate_synthetic_data(symbol, period)

print(f"📅 Período: {data.index[0].strftime('%Y-%m-%d')} a {data.index[-1].strftime('%Y-%m-%d')}")
print(f"📊 Total de registros: {len(data)}")

In [ ]:
# Visualizar os primeiros registros
print("📋 Primeiros 10 registros:")
display(data.head(10))

print("\n📋 Últimos 10 registros:")
display(data.tail(10))

In [ ]:
# Informações básicas sobre os dados
print("📊 Informações dos dados:")
print(data.info())

print("\n📈 Estatísticas descritivas:")
display(data.describe())

In [ ]:
# Visualizar a série temporal
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle(f'Série Temporal - {symbol}', fontsize=16)

# Plot 1: Preço de fechamento
axes[0, 0].plot(data.index, data['Close'], 'b-', linewidth=1)
axes[0, 0].set_title('Preço de Fechamento')
axes[0, 0].set_xlabel('Data')
axes[0, 0].set_ylabel('Preço ($)')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Volume
axes[0, 1].plot(data.index, data['Volume'], 'g-', linewidth=1)
axes[0, 1].set_title('Volume')
axes[0, 1].set_xlabel('Data')
axes[0, 1].set_ylabel('Volume')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Retornos diários
returns = data['Close'].pct_change().dropna()
axes[1, 0].plot(data.index[1:], returns, 'r-', linewidth=1)
axes[1, 0].set_title('Retornos Diários')
axes[1, 0].set_xlabel('Data')
axes[1, 0].set_ylabel('Retorno')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Distribuição dos retornos
axes[1, 1].hist(returns, bins=50, alpha=0.7, edgecolor='black')
axes[1, 1].set_title('Distribuição dos Retornos')
axes[1, 1].set_xlabel('Retorno')
axes[1, 1].set_ylabel('Frequência')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estatísticas dos retornos
print("📊 Estatísticas dos retornos diários:")
print(f"Média: {returns.mean():.6f}")
print(f"Desvio padrão: {returns.std():.6f}")
print(f"Skewness: {returns.skew():.6f}")
print(f"Kurtosis: {returns.kurtosis():.6f}")

## 2. 🔧 Preparação dos Dados

Agora vamos preparar os dados para treinamento dos modelos RNN.

In [ ]:
# Configurações para preparação dos dados
sequence_length = 60  # Número de dias para prever o próximo

print(f"🔧 Preparando sequências com {sequence_length} dias de histórico...")

# Preparar sequências
X, y = loader.prepare_sequences(data, sequence_length=sequence_length)

print(f"✅ Dados preparados!")
print(f"📊 Shape dos dados de entrada (X): {X.shape}")
print(f"📊 Shape dos dados de saída (y): {y.shape}")
print(f"📊 Cada sequência tem {sequence_length} dias de histórico")
print(f"📊 Total de amostras: {len(X)}")

In [ ]:
# Dividir dados em treino, validação e teste
X_train, X_val, X_test, y_train, y_val, y_test = loader.split_data(X, y)

print("📊 Divisão dos dados:")
print(f"   Treino: {X_train.shape[0]} amostras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"   Validação: {X_val.shape[0]} amostras ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"   Teste: {X_test.shape[0]} amostras ({X_test.shape[0]/len(X)*100:.1f}%)")

# Visualizar algumas sequências
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Exemplos de Sequências de Treinamento', fontsize=16)

for i in range(4):
    row = i // 2
    col = i % 2
    
    # Plotar sequência
    axes[row, col].plot(X_train[i], 'b-', label='Sequência de entrada')
    axes[row, col].axhline(y=y_train[i], color='r', linestyle='--', label='Valor alvo')
    axes[row, col].set_title(f'Sequência {i+1}')
    axes[row, col].set_xlabel('Dias')
    axes[row, col].set_ylabel('Preço Normalizado')
    axes[row, col].legend()
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 🎯 Treinamento de Modelos

Agora vamos treinar diferentes modelos RNN (LSTM e GRU) e comparar suas performances.

In [ ]:
# Configurações dos modelos
model_configs = {
    'lstm': {
        'hidden_size': 50,
        'num_layers': 2,
        'dropout': 0.2
    },
    'gru': {
        'hidden_size': 50,
        'num_layers': 2,
        'dropout': 0.2
    }
}

# Configurações de treinamento
train_config = {
    'epochs': 30,  # Reduzido para demonstração
    'batch_size': 32,
    'learning_rate': 0.001
}

print("🎯 Configurações de treinamento:")
print(f"   Épocas: {train_config['epochs']}")
print(f"   Batch size: {train_config['batch_size']}")
print(f"   Learning rate: {train_config['learning_rate']}")
print(f"   Modelos: {list(model_configs.keys())}")

In [ ]:
# Dicionário para armazenar resultados
results = {}

# Treinar cada modelo
for model_type, config in model_configs.items():
    print(f"\n🚀 Treinando modelo {model_type.upper()}...")
    print(f"   Hidden size: {config['hidden_size']}")
    print(f"   Num layers: {config['num_layers']}")
    print(f"   Dropout: {config['dropout']}")
    
    # Criar trainer
    trainer = TimeSeriesTrainer(
        model_type=model_type,
        input_size=1,
        hidden_size=config['hidden_size'],
        num_layers=config['num_layers'],
        output_size=1,
        dropout=config['dropout']
    )
    
    # Preparar dataloaders
    train_loader, val_loader = trainer.prepare_data_loaders(
        X_train, y_train, X_val, y_val, batch_size=train_config['batch_size']
    )
    
    # Treinar modelo
    train_losses, val_losses, train_metrics, val_metrics = trainer.train(
        train_loader, val_loader,
        epochs=train_config['epochs'],
        learning_rate=train_config['learning_rate'],
        patience=10
    )
    
    # Avaliar modelo
    predictions, actuals, test_metrics = trainer.evaluate(
        X_test, y_test, loader.scaler
    )
    
    # Armazenar resultados
    results[model_type] = {
        'trainer': trainer,
        'predictions': predictions,
        'actuals': actuals,
        'test_metrics': test_metrics,
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_metrics': train_metrics,
        'val_metrics': val_metrics
    }
    
    print(f"✅ Modelo {model_type.upper()} treinado com sucesso!")
    print(f"   R² Score: {test_metrics['r2']:.4f}")
    print(f"   RMSE: {test_metrics['rmse']:.4f}")
    print(f"   MAPE: {test_metrics['mape']:.2f}%")

## 4. 📈 Comparação de Performance

Vamos comparar a performance dos diferentes modelos treinados.

In [ ]:
# Criar DataFrame de comparação
comparison_data = {}
for model_type, result in results.items():
    comparison_data[model_type] = result['test_metrics']

comparison_df = pd.DataFrame(comparison_data).T
print("📊 Comparação de Performance dos Modelos:")
display(comparison_df)

# Identificar melhor modelo
best_model = max(comparison_data.keys(), key=lambda x: comparison_data[x]['r2'])
print(f"\n🏆 Melhor modelo: {best_model.upper()} (R² = {comparison_data[best_model]['r2']:.4f})")

In [ ]:
# Visualizar comparação
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Comparação de Modelos', fontsize=16)

metrics = ['mse', 'rmse', 'mae', 'r2', 'mape']
model_names = list(comparison_data.keys())

for i, metric in enumerate(metrics):
    row = i // 3
    col = i % 3
    
    values = [comparison_data[model][metric] for model in model_names]
    
    if metric == 'r2':
        # Para R², valores mais altos são melhores
        colors = ['green' if v > 0.8 else 'orange' if v > 0.6 else 'red' for v in values]
    else:
        # Para outras métricas, valores mais baixos são melhores
        colors = ['green' if v < np.mean(values) else 'orange' if v < np.mean(values) * 1.5 else 'red' for v in values]
    
    bars = axes[row, col].bar(model_names, values, color=colors, alpha=0.7)
    axes[row, col].set_title(f'{metric.upper()}')
    axes[row, col].set_ylabel(metric.upper())
    
    # Adicionar valores nas barras
    for bar, value in zip(bars, values):
        height = bar.get_height()
        axes[row, col].text(bar.get_x() + bar.get_width()/2., height,
                          f'{value:.4f}', ha='center', va='bottom')

# Remover subplot extra
fig.delaxes(axes[1, 2])

plt.tight_layout()
plt.show()

## 5. 🏆 Análise do Melhor Modelo

Vamos fazer uma análise detalhada do melhor modelo.

In [ ]:
# Obter resultados do melhor modelo
best_result = results[best_model]

print(f"🏆 Análise detalhada do modelo {best_model.upper()}")
print("=" * 50)

# Métricas finais
metrics = best_result['test_metrics']
print(f"📊 Métricas de Teste:")
print(f"   R² Score: {metrics['r2']:.4f}")
print(f"   RMSE: {metrics['rmse']:.4f}")
print(f"   MAE: {metrics['mae']:.4f}")
print(f"   MAPE: {metrics['mape']:.2f}%")
print(f"   Precisão: {100 - metrics['mape']:.2f}%")

In [ ]:
# Plotar histórico de treinamento
plot_training_history(
    best_result['train_losses'],
    best_result['val_losses'],
    best_result['train_metrics'],
    best_result['val_metrics'],
    symbol,
    best_model
)

In [ ]:
# Plotar predições vs valores reais
plot_predictions(
    best_result['actuals'],
    best_result['predictions'],
    symbol,
    best_model
)

In [ ]:
# Análise de resíduos
print("🔍 Análise de Resíduos:")
residual_stats = analyze_residuals(
    best_result['actuals'],
    best_result['predictions']
)

## 6. 🔮 Previsões Futuras

Agora vamos usar o melhor modelo para fazer previsões futuras.

In [ ]:
# Configurações para previsões futuras
days_ahead = 30
last_sequence = X_test[-1]  # Última sequência do conjunto de teste

print(f"🔮 Fazendo previsões para os próximos {days_ahead} dias...")
print(f"📊 Usando a última sequência de {sequence_length} dias como base")

# Fazer previsões
future_predictions = best_result['trainer'].predict_future(
    last_sequence, loader.scaler, days_ahead
)

print(f"✅ Previsões geradas com sucesso!")
print(f"📈 Último preço conhecido: ${best_result['actuals'][-1]:.2f}")
print(f"🔮 Primeira previsão: ${future_predictions[0]:.2f}")
print(f"🔮 Última previsão: ${future_predictions[-1]:.2f}")

In [ ]:
# Mostrar previsões detalhadas
print("📋 Previsões detalhadas:")
print("Dia | Preço Previsto | Variação")
print("-" * 35)

last_known_price = best_result['actuals'][-1]
for i, pred in enumerate(future_predictions):
    variation = ((pred - last_known_price) / last_known_price) * 100
    print(f"{i+1:3d} | ${pred:8.2f} | {variation:+6.2f}%")
    
    # Mostrar apenas os primeiros 15 dias para não poluir a saída
    if i >= 14:
        print(f"... | ... | ...")
        print(f"{days_ahead:3d} | ${future_predictions[-1]:8.2f} | {((future_predictions[-1] - last_known_price) / last_known_price) * 100:+6.2f}%")
        break

In [ ]:
# Plotar previsões futuras
plot_future_predictions(
    data,
    future_predictions,
    symbol,
    best_model,
    days_ahead
)

## 7. 📋 Relatório Final

Vamos gerar um relatório final com todos os resultados.

In [ ]:
# Gerar relatório de performance
report = create_performance_report(
    best_result['test_metrics'],
    symbol,
    best_model
)

print("📋 RESUMO FINAL")
print("=" * 50)
print(f"📊 Símbolo analisado: {symbol}")
print(f"📅 Período: {data.index[0].strftime('%Y-%m-%d')} a {data.index[-1].strftime('%Y-%m-%d')}")
print(f"📈 Total de registros: {len(data)}")
print(f"🔧 Sequência de entrada: {sequence_length} dias")
print(f"🎯 Modelos treinados: {list(results.keys())}")
print(f"🏆 Melhor modelo: {best_model.upper()}")
print(f"📊 R² Score: {comparison_data[best_model]['r2']:.4f}")
print(f"📊 Precisão: {100 - comparison_data[best_model]['mape']:.2f}%")
print(f"🔮 Previsões futuras: {days_ahead} dias")

In [ ]:
# Comparação final de todos os modelos
print("\n📊 COMPARAÇÃO FINAL DE TODOS OS MODELOS")
print("=" * 50)

for model_type, result in results.items():
    metrics = result['test_metrics']
    print(f"\n{model_type.upper()}:")
    print(f"   R² Score: {metrics['r2']:.4f}")
    print(f"   RMSE: {metrics['rmse']:.4f}")
    print(f"   MAPE: {metrics['mape']:.2f}%")
    print(f"   Precisão: {100 - metrics['mape']:.2f}%")
    
    if model_type == best_model:
        print(f"   🏆 MELHOR MODELO!")

print(f"\n✅ Análise concluída com sucesso!")
print(f"📁 Verifique as pastas 'plots/', 'models/' e 'reports/' para os resultados salvos.")

## 🎯 Conclusões

Neste notebook, realizamos uma análise completa de séries temporais usando Redes Neurais Recorrentes:

### 📊 Principais Resultados:
- **Carregamento de dados**: Dados de ações carregados com sucesso
- **Preparação**: Sequências temporais preparadas adequadamente
- **Treinamento**: Modelos LSTM e GRU treinados com sucesso
- **Comparação**: Performance dos modelos avaliada e comparada
- **Previsões**: Previsões futuras geradas com o melhor modelo

### 🔧 Aspectos Técnicos:
- **Normalização**: Dados normalizados usando MinMaxScaler
- **Sequências**: Sequências de 60 dias para prever o próximo dia
- **Divisão**: Dados divididos em treino (80%), validação (10%) e teste (10%)
- **Métricas**: R², RMSE, MAE e MAPE utilizadas para avaliação

### 🚀 Próximos Passos:
- Experimentar com diferentes arquiteturas de rede
- Ajustar hiperparâmetros para melhorar performance
- Adicionar mais features (volume, indicadores técnicos)
- Implementar ensemble de modelos
- Usar dados de múltiplas ações simultaneamente

### ⚠️ Limitações:
- Previsões de séries temporais são inerentemente difíceis
- Mercados financeiros são altamente voláteis e imprevisíveis
- Modelos devem ser re-treinados periodicamente
- Resultados passados não garantem performance futura

**Nota**: Este notebook é para fins educacionais. Não use estas previsões para decisões de investimento sem análise adicional e consulta a profissionais qualificados.